In [1]:
import sys
sys.path.append("..")

%load_ext autoreload
%autoreload complete

In [2]:
import matplotlib.pyplot as plt
import xarray as xr

from project.test_pendulum import *
from project.util import *

In [3]:
def make_dataset():
    tt, xx, yy = run_oscillator(t_end=200)

    ds = xr.Dataset({
        "x": (["time"], yy[0,:]),
        "v": (["time"], yy[1,:]),
    }, coords={
        "time": tt
    })

    return ds

ds = make_dataset()
ds

G: [[1.         1.10517092]
 [0.99004983 0.998002  ]]
L: [[ 0.    1.  ]
 [-0.1  -0.02]]
Eigenvalues of L: [-0.01+0.31606961j -0.01-0.31606961j]


<xarray.Dataset>
Dimensions:  (time: 2000)
Coordinates:
  * time     (time) float64 0.0 0.1 0.2 0.3 0.4 ... 199.6 199.7 199.8 199.9
Data variables:
    x        (time) float64 1.009 1.029 0.9899 1.004 ... 0.3521 0.3397 0.343
    v        (time) float64 -0.01117 -0.004639 -0.01977 ... -0.03324 -0.02925

In [4]:
data = stack_state(ds)
data

<xarray.DataArray '' (state: 2, time: 2000)>
array([[ 1.00912983,  1.02894905,  0.9899048 , ...,  0.35209139,
         0.33967726,  0.34296409],
       [-0.01116584, -0.0046391 , -0.01976569, ..., -0.0409228 ,
        -0.03323912, -0.02925258]])
Coordinates:
  * time     (time) float64 0.0 0.1 0.2 0.3 0.4 ... 199.6 199.7 199.8 199.9
  * state    (state) object MultiIndex
  * field    (state) object 'x' 'v'

In [5]:
taus = data["time"].diff("time").values
assert all(np.isclose(taus[0], taus))  # must have uniform dt
tau0 = taus[0].item()
tau0

0.1

In [6]:
data_t0 = data.isel(time=slice(None, -1))
data_t0

<xarray.DataArray '' (state: 2, time: 1999)>
array([[ 1.00912983,  1.02894905,  0.9899048 , ...,  0.36123355,
         0.35209139,  0.33967726],
       [-0.01116584, -0.0046391 , -0.01976569, ..., -0.03306743,
        -0.0409228 , -0.03323912]])
Coordinates:
  * time     (time) float64 0.0 0.1 0.2 0.3 0.4 ... 199.5 199.6 199.7 199.8
  * state    (state) object MultiIndex
  * field    (state) object 'x' 'v'

In [7]:
data_tau0 = data.isel(time=slice(1, None))
data_tau0

<xarray.DataArray '' (state: 2, time: 1999)>
array([[ 1.02894905,  0.9899048 ,  1.00425625, ...,  0.35209139,
         0.33967726,  0.34296409],
       [-0.0046391 , -0.01976569, -0.02681867, ..., -0.0409228 ,
        -0.03323912, -0.02925258]])
Coordinates:
  * time     (time) float64 0.1 0.2 0.3 0.4 0.5 ... 199.6 199.7 199.8 199.9
  * state    (state) object MultiIndex
  * field    (state) object 'x' 'v'

In [8]:
C_tau0 = estimate_cov(data_tau0, data_t0)
C_t0 = estimate_cov(data_t0, data_t0)
G = xr.dot(C_tau0, inverse(C_t0), dims=[])
G

<xarray.DataArray (state1: 2, state2: 2)>
array([[ 1.00068273, -0.0025188 ],
       [-0.0025188 ,  1.00302103]])
Coordinates:
  * state1   (state1) object MultiIndex
  * field1   (state1) <U1 'x' 'v'
  * state2   (state2) object MultiIndex
  * field2   (state2) <U1 'x' 'v'